# Segmenting and Clustering Neighborhoods in the city of Toronto

In [71]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis


Read url to get Wikipedia Table, call it canada

In [72]:
url='http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
canada=pd.read_html(url, header=0)[0]
print(canada.shape)
canada.head()

(287, 3)


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove cells where Borough ist not assigned

In [73]:
canada=canada[canada.Borough!='Not assigned']
print(canada.shape)
canada.head()

(210, 3)


,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [74]:
canada.loc[canada.Neighborhood=='Not assigned','Neighborhood']=canada.loc[canada.Neighborhood=='Not assigned','Borough']
canada.head(200)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


List all Neighborhoods that belong to one Postcode. Rename dataframe df now

In [75]:
#df=pd.DataFrame(canada.groupby(['Postcode','Borough'])['Neighborhood'].apply(list)).reset_index()
df=canada.groupby(['Postcode','Borough'])['Neighborhood'].agg(lambda x: ','.join(set(x))).reset_index()
df.head(200)

(103, 3)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,West Hill,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview,East Birchmount Park,Kennedy Park"
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [76]:
print(df.shape)


(103, 3)
